# (4주차) 3월23일

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최규빈

### 강의영상 

### imports 

In [2]:
import tensorflow as tf 
import tensorflow.experimental.numpy as tnp 

In [16]:
import matplotlib.pyplot as plt 

In [3]:
tnp.experimental_enable_numpy_behavior()

### 미분 

#### tf.GradientTape() 사용방법

`-` 예제9: 카페예제로 돌아오자. 

In [6]:
x=tnp.array([20.1, 22.2, 22.7, 23.3, 24.4, 25.1, 26.2, 27.3, 28.4, 30.4])
x

<tf.Tensor: shape=(10,), dtype=float64, numpy=array([20.1, 22.2, 22.7, 23.3, 24.4, 25.1, 26.2, 27.3, 28.4, 30.4])>

In [15]:
tnp.random.seed(43052) 
y= 10.2+ x*2.2 + tnp.random.randn(10) 
y

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([54.98269924, 60.27348365, 61.27621687, 60.53495888, 62.9770905 ,
       66.32168996, 66.87781372, 71.0050025 , 72.63837337, 77.11143943])>

In [19]:
beta0= tf.Variable(9.0) 
beta1= tf.Variable(2.0)  

In [23]:
with tf.GradientTape(persistent=True) as tape: 
    loss=sum((y-beta0-beta1*x)**2)

In [27]:
tape.gradient(loss,beta0),tape.gradient(loss,beta1) 

(<tf.Tensor: shape=(), dtype=float32, numpy=-127.597534>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-3214.2532>)

`-` 예제10: 카페예제의 매트릭스 버전 

In [39]:
X= tnp.array([1]*10+ [20.1, 22.2, 22.7, 23.3, 24.4, 25.1, 26.2, 27.3, 28.4, 30.4]).reshape(2,10).T
X

<tf.Tensor: shape=(10, 2), dtype=float64, numpy=
array([[ 1. , 20.1],
       [ 1. , 22.2],
       [ 1. , 22.7],
       [ 1. , 23.3],
       [ 1. , 24.4],
       [ 1. , 25.1],
       [ 1. , 26.2],
       [ 1. , 27.3],
       [ 1. , 28.4],
       [ 1. , 30.4]])>

In [40]:
beta_true = tnp.array([[10.2],[2.2]])
beta_true

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[10.2],
       [ 2.2]])>

In [42]:
tnp.random.seed(43052) 
y= X@beta_true + tnp.random.randn(10).reshape(10,1)
y

<tf.Tensor: shape=(10, 1), dtype=float64, numpy=
array([[54.98269924],
       [60.27348365],
       [61.27621687],
       [60.53495888],
       [62.9770905 ],
       [66.32168996],
       [66.87781372],
       [71.0050025 ],
       [72.63837337],
       [77.11143943]])>

In [43]:
beta = tnp.array([[9.0],[2.0]])
beta

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[9.],
       [2.]])>

In [51]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(beta)
    yhat = X@beta 
    loss=(y-yhat).T @ (y-yhat) 

In [52]:
tape.gradient(loss,beta)

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[ -127.59753624],
       [-3214.25306574]])>

`-` 예제11: 위의 예제에서 이론적인 $\boldsymbol{\beta}$의 최적값을 찾아보고 (즉 $\boldsymbol{\hat\beta}$을 찾고) 그 지점에서 loss의 미분값(=접선의 기울기)를 구하라. 결과가 $\bf{0}$인지 확인하라. (단 ${\bf 0}$은 길이가 2이고 각 원소가 0인 벡터)

## 경사하강법 

### 최적화문제 

`-` $loss=(\frac{1}{2}\beta-1)^2$를 최소하는 $\beta$를 컴퓨터를 활용하여 구하는 문제를 생각해보자.

### 방법1: grid search

#### 알고리즘 

#### 구현코드 

`-` tnp.argmin

`-` 풀이 

`-` 최소지점 주변값과 비교 

#### 그리드서치의 문제점 

`-` 비판1: [-10,10]이외에 해가 존재하면? 
- 이 예제의 경우는 운좋게 [-10,10]에서 해가 존재했음
- 하지만 임의의 고정된 $x,y$에 대하여 $loss(\beta)=(x\beta-y)^2$ 의 형태의 해가 항상 [-10,10]에서 존재한다는 보장은 없음
- 해결책: 더 넓게 많은 범위를 탐색하자? 

`-` 비판2: 효율적이지 않음
- 알고리즘을 요약하면 결국 -10부터 10까지 작은 간격으로 조금씩 이동하며 loss를 조사하는 것이 grid search의 아이디어 
- $\to$ 생각해보니까 $\beta=2$인 순간 $loss=(\frac{1}{2}\beta-1)^2=0$이 되어서 이것보다 작은 최소값은 존재하지 않는다(제곱은 항상 양수이어야 하므로)
- $\to$ 따라서 $\beta=2$ 이후로는 탐색할 필요가 없다 

### 방법2: gradient descent

#### 알고리즘!

(1) 임의의 초기값을 선정하고 loss를 계산한다. 
 - $\beta=-5 \to loss(-5)=(-5/2-1)^2=$

(2) 임의의 초기값에서 좌우로 약간씩 이동해보고 loss를 계산한다.
 - 왼쪽으로 이동: $\beta=-5.01,\quad loss(5.01)=$
 - 오른쪽으로 이동: $\beta=-4.99, \quad loss(-4.99)=$ 

(3) (2)의 결과를 보고 어느쪽으로 이동하는것이 유리한지 따져보고 `유리한 방향으로` 이동한다. 
 - $\beta=-4.99$ 로 이동 


(4) (2)-(3) 의 과정을 반복한다. **왼쪽/오른쪽 모두 가봐도 유리한 지점이 없다면 알고리즘을 멈춘다.**

#### 잠깐 알고리즘 감상 

`-` 정지조건?

`-` [-10,10] 이외에 해가 존재? 

`-` $\beta=2$를 찾았다면? 

`-` 알고리즘해석 
- (2)의 과정: 
- (3)의 과정: 

> Note: 이처럼 손실함수의 기울기(=경사)를 계산하여 점차적으로 가중치를 업데이트 하는 방식을 경사하강법이라고 부른다. 

#### 왼쪽/오른쪽중에 어디로 갈지 어떻게 판단하는 과정을 수식화?

`-` 미분계수가 + - ? 

`-` 수식화 

#### hoxy, 알고리즘을 좀 개선할수 있을까?

`-` 왜 0.01씩? 

`-` $\beta=-10$ 일 경우의 접선의 기울기? $\beta=-4$ 일때 접선의 기울기?

`-` 아이디어

`-` 수식화 

$$\beta_{next} \leftarrow \beta_{old} -\alpha \left[\frac{\partial}{\partial \beta} loss(\beta)\right]_{\beta=\beta_{old}}$$

- 아까 수식이랑 좀 다르다? 
- $\alpha$의 조건 
- $\alpha$의 의미 
- $\beta=-10$ 에서 $0.01$만큼 움직이기 위한 $\alpha$의 설정 

#### 구현코드 